In [1]:
from enum import IntEnum
import random
import json
import pickle 
import numpy as np
import nltk 
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize

lemmatizer=WordNetLemmatizer()

intents= json.loads(open('intents.json').read())

words=[]
classes=[]
documents=[]
ignore_letters=['?','!','.',',']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word = nltk.word_tokenize(pattern)
        words.extend(word)
        documents.append((word, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))



training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    word_patterns = doc[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training,dtype=object)
print(training)

train_x = list(training[:, 0])
train_y = list(training[:, 1])

# print(train_y)
# print(train_y)

# list_x= list([x for x in training[:,0] if x != []])
# list_y= list(training[:, 1])
# print(list_x)
# print(len(list_y))


[[list([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
  list([1, 0, 0])]
 [list([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0])
  list([0, 0, 1])]
 [list([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0])
  list([0, 1, 0])]
 [list([1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
  list([1, 0, 0])]
 [list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
  list([0, 1, 0])]
 [list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
  list([0, 1, 0])]
 [list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
  list([0, 1, 0])]
 [list

In [2]:
X =np.array(train_x,dtype="float32")
Y = np.array(train_y)


In [3]:
import tensorflow as tf
from keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
from keras.models import Sequential
from keras.models import load_model


In [4]:
model= Sequential()
model.add(Dense(128, input_shape=(np.array(train_x[0]).shape), activation='ReLU'))
model.add(Dropout(0.5))
model.add(Dense(64,  activation='ReLU'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
sgd=SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
hey=model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save("cb.h5",hey)
model.summary()
print("done")

Epoch 1/200


C:\Users\laksh\anaconda3\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


4/4 [==============================] - 1s 4ms/step - loss: 1.1268 - accuracy: 0.3000
Epoch 2/200
4/4 [==============================] - 0s 4ms/step - loss: 1.1456 - accuracy: 0.3500
Epoch 3/200
4/4 [==============================] - 0s 4ms/step - loss: 1.1087 - accuracy: 0.3000
Epoch 4/200
4/4 [==============================] - 0s 4ms/step - loss: 1.0671 - accuracy: 0.5500
Epoch 5/200
4/4 [==============================] - 0s 4ms/step - loss: 1.0084 - accuracy: 0.6500
Epoch 6/200
4/4 [==============================] - 0s 5ms/step - loss: 0.9367 - accuracy: 0.6000
Epoch 7/200
4/4 [==============================] - 0s 5ms/step - loss: 1.0471 - accuracy: 0.5000
Epoch 8/200
4/4 [==============================] - 0s 3ms/step - loss: 0.9218 - accuracy: 0.6000
Epoch 9/200
4/4 [==============================] - 0s 5ms/step - loss: 0.8335 - accuracy: 0.7000
Epoch 10/200
4/4 [==============================] - 0s 4ms/step - loss: 0.8584 - accuracy: 0.7000
Epoch 11/200
4/4 [=======================

4/4 [==============================] - 0s 5ms/step - loss: 0.0262 - accuracy: 1.0000
Epoch 85/200
4/4 [==============================] - 0s 5ms/step - loss: 0.0951 - accuracy: 0.9500
Epoch 86/200
4/4 [==============================] - 0s 4ms/step - loss: 0.0222 - accuracy: 1.0000
Epoch 87/200
4/4 [==============================] - 0s 5ms/step - loss: 0.0400 - accuracy: 1.0000
Epoch 88/200
4/4 [==============================] - 0s 5ms/step - loss: 0.0286 - accuracy: 1.0000
Epoch 89/200
4/4 [==============================] - 0s 5ms/step - loss: 0.0510 - accuracy: 1.0000
Epoch 90/200
4/4 [==============================] - 0s 5ms/step - loss: 0.0550 - accuracy: 1.0000
Epoch 91/200
4/4 [==============================] - 0s 5ms/step - loss: 0.0924 - accuracy: 1.0000
Epoch 92/200
4/4 [==============================] - 0s 5ms/step - loss: 0.1133 - accuracy: 0.9500
Epoch 93/200
4/4 [==============================] - 0s 5ms/step - loss: 0.0124 - accuracy: 1.0000
Epoch 94/200
4/4 [===============

4/4 [==============================] - 0s 4ms/step - loss: 0.0110 - accuracy: 1.0000
Epoch 168/200
4/4 [==============================] - 0s 4ms/step - loss: 0.0066 - accuracy: 1.0000
Epoch 169/200
4/4 [==============================] - 0s 4ms/step - loss: 0.0305 - accuracy: 1.0000
Epoch 170/200
4/4 [==============================] - 0s 4ms/step - loss: 0.0092 - accuracy: 1.0000
Epoch 171/200
4/4 [==============================] - 0s 5ms/step - loss: 0.0187 - accuracy: 1.0000
Epoch 172/200
4/4 [==============================] - 0s 5ms/step - loss: 0.0091 - accuracy: 1.0000
Epoch 173/200
4/4 [==============================] - 0s 5ms/step - loss: 0.0033 - accuracy: 1.0000
Epoch 174/200
4/4 [==============================] - 0s 5ms/step - loss: 0.0091 - accuracy: 1.0000
Epoch 175/200
4/4 [==============================] - 0s 4ms/step - loss: 0.0033 - accuracy: 1.0000
Epoch 176/200
4/4 [==============================] - 0s 5ms/step - loss: 0.0038 - accuracy: 1.0000
Epoch 177/200
4/4 [=====

In [5]:
# hey.predict(train_x[0])
yes=np.array(train_x[0])
no=np.reshape(train_x[0],(38,128))
# train_x= (np.array(train_x[0]).reshape(38, 128)).astype("float32") / 255
# model.predict(np.array(train_x[0]))

ValueError: cannot reshape array of size 38 into shape (38,128)

In [ ]:
yes=np.array(train_x[0])
print(yes.shape)
# print(np.array(train_x[0]).shape)
# newarr = yes.reshape(0,40)


In [ ]:
model=load_model('cb.h5')

In [ ]:
yes=np.array(train_x[0])
print(yes)

In [ ]:
print(np.array[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0].transpose())